## CLEFIA CPA Attack

```
Conduct Correlation Power Analysis attacks at the first, second and last rounds of CLEFIA-128 encryption. 

Authors: Arjun Menon V (ee18b104), Akilesh Kannan (ee18b122)
```

Assignment 2, Secure Processor Microarchitecture

August 2022

In [48]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress

In [49]:
path_to_data = '../data/'

### Where should I take the Intermediate Result?

At this point, we have two choices on the location of the intermediate result:

- Output of the SBox opertion, in the GFN4 network
- Output of the F0/1 operation in the GFN4 network

The output of each SBox is an 8-bit word, and hence we can perform key search with $2^8$ guesses for each of the 16 Bytes of the Key. On the contrary, the output of the F functions are 32-bits wide and the search space involves $2^{32}$ possibilities for each of the 4 Words of the key. 

However, if the implementation uses a look-up table for the entire F function, without evaluating the intermediate SBox result, the former attack scheme will not work as there won't be any correlation between the intermediate result and the power trace. Hence this is a weaker attack, despite having lower online complexity. 

**Approach:**

- Try the SBox attack first and see if correlations are got with sufficient confidence
- Validate the results against the PT-CT pair provided
- If the previous attack fails, try the F0/1 attack

In [50]:
def f0(inp):
    
    tf00 = np.load(path_to_data+"tf00.npy", "r")
    tf01 = np.load(path_to_data+"tf01.npy", "r")
    tf02 = np.load(path_to_data+"tf02.npy", "r")
    tf03 = np.load(path_to_data+"tf03.npy", "r")
    
    out = tf00[inp[0]] ^ tf01[inp[1]] ^ tf02[inp[2]] ^ tf03[inp[3]]
    return out

def f1(inp):
    
    tf10 = np.load(path_to_data+"tf10.npy", "r")
    tf11 = np.load(path_to_data+"tf11.npy", "r")
    tf12 = np.load(path_to_data+"tf12.npy", "r")
    tf13 = np.load(path_to_data+"tf13.npy", "r")
    
    out = tf10[inp[0]] ^ tf11[inp[1]] ^ tf12[inp[2]] ^ tf13[inp[3]]
    return out

def intermediate(pt, rk0, rk1, wk0, wk1):
    # Assumption: pt is an array of 32-bit integers with 4 elements; rk's and wk's are 32-b ints
        
    # Ensure correct bit-width match!
    out0 = f0(rk0 ^ pt[0])
    out1 = pt[1] ^ wk0
    out2 = f1(rk1 ^ pt[2])
    out3 = pt[3] ^ wk1
    out = [out0, out1, out2, out3]
    return out

In [51]:
textin_arr = np.load("../EE18B104_EE18B122/trace_002/textin_array.npy", "r")
pt = textin_arr[0][0:4]
rk0 = np.random.randint(-128, 128, size= 1).astype(np.int32)
wk0 = np.random.randint(-128, 128, size= 1).astype(np.int32)
rk1 = np.random.randint(-128, 128, size= 1).astype(np.int32)
wk1 = np.random.randint(-128, 128, size= 1).astype(np.int32)
iv = intermediate(pt, rk0, rk1, wk0, wk1)
print(iv)

IndexError: index 1 is out of bounds for axis 0 with size 1

In [19]:
help(np.random.randint)

Help on built-in function randint:

randint(...) method of numpy.random.mtrand.RandomState instance
    randint(low, high=None, size=None, dtype=int)
    
    Return random integers from `low` (inclusive) to `high` (exclusive).
    
    Return random integers from the "discrete uniform" distribution of
    the specified dtype in the "half-open" interval [`low`, `high`). If
    `high` is None (the default), then results are from [0, `low`).
    
    .. note::
        New code should use the ``integers`` method of a ``default_rng()``
        instance instead; please see the :ref:`random-quick-start`.
    
    Parameters
    ----------
    low : int or array-like of ints
        Lowest (signed) integers to be drawn from the distribution (unless
        ``high=None``, in which case this parameter is one above the
        *highest* such integer).
    high : int or array-like of ints, optional
        If provided, one above the largest (signed) integer to be drawn
        from the distributi